In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree

from joblib import Parallel, delayed

In [2]:
from QuadraticConstraintModel import get_leaf_samples

from QuadraticConstraintModel import constrained_optimization_gurobi

from QuadraticConstraintModel import predict_from_COF

from QuadraticConstraintModel import  get_h_from_COF

In [3]:
# Function to load DataSet
def load_dataset(file_path, num_attributes=2, num_classes=2):
    data = pd.read_csv(file_path)
    X = data.iloc[:, 0 :  num_attributes].values
    y = data.iloc[:,  num_attributes:  num_attributes + num_classes].values
    # y = data.iloc[:, 9:10].values
    return X, y

In [79]:
def normalized_root_mean_square_error(y_true, y_pred):
    """
    Computes the Normalized Root Mean Square Error (NRMSE) between y_true and y_pred.
    If the range of y_true is zero, it normalizes by the number of samples * outputs.

    Parameters:
        y_true (np.ndarray): Ground truth values, shape (n_samples, n_outputs)
        y_pred (np.ndarray): Predicted values, shape (n_samples, n_outputs)

    Returns:
        float: NRMSE value
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Compute RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    
    # Compute range
    y_range = np.max(y_true) - np.min(y_true)
    
    if y_range != 0:
        # Normalize by range
        return rmse / y_range
    else:
        # Normalize by n_samples * n_outputs
        n_samples, n_outputs = y_true.shape
        return np.sqrt(np.sum((y_true - y_pred) ** 2) / (n_samples * n_outputs))


In [80]:
def process_leaf(leaf_id, indices, X_train, y_train, feature_names, optimizer):
    X_leaf = X_train[indices]
    y_leaf = y_train[indices]
    
    # Choose optimizer
    if optimizer == "gurobi":
        M, m0, h = constrained_optimization_gurobi(X_leaf, y_leaf)
    elif optimizer == "gurobi_MSE":
        M, m0, h = constrained_optimization_MSE_gurobi(X_leaf, y_leaf)
    elif optimizer == "least_squares":
        M, m0, h = least_squares_solution(X_leaf, y_leaf)
    elif optimizer == "gurobi_l2":
        # print("Optimizing with L2 regularization")
        M, m0, h = constrained_optimization_regularization_gurobi(X_leaf, y_leaf)
    elif optimizer == "gurobi_MSE_l2":
        # print("Optimizing with MSE L2 regularization")
        M, m0, h = constrained_optimization_MSE_regularization_gurobi(X_leaf, y_leaf)
    else:
        M, m0, h = constrained_optimization(X_leaf, y_leaf)
    
    # Build model info
    model = {
        "leaf_id": leaf_id,
        "CO_Model": {'M': M, 'm0': m0, 'h': h},
        "no_samples": len(indices),
        "indices": indices,
        "bounds": {
            feature_names[i]: (X_leaf[:, i].min(), X_leaf[:, i].max())
            for i in range(X_leaf.shape[1])
        }
    }
    return model


def train_COF_on_leaves_parallel(X_train, y_train, tree, feature_names=None, optimizer="gurobi", n_jobs=-1):
    """
    Train constrained optimization models on tree leaves in parallel.

    Parameters:
        X_train, y_train : np.ndarray
        tree : fitted sklearn tree
        feature_names : list of feature names (optional)
        optimizer : {"gurobi", "CVXPY + SCS"}
        n_jobs : number of parallel workers (-1 = all cores)
    """
    leaf_samples = get_leaf_samples(tree, X_train)

    if feature_names is None:
        feature_names = [f"feature_{i}" for i in range(X_train.shape[1])]

    # Run leaf computations in parallel
    tree_extracted_info = Parallel(n_jobs=n_jobs)(
        delayed(process_leaf)(leaf_id, indices, X_train, y_train, feature_names, optimizer)
        for leaf_id, indices in leaf_samples.items()
    )

    return tree_extracted_info


In [75]:
sys_name = "navigation_old"
n_samples = 500000
X, y = load_dataset(f"Dataset/{sys_name}/{sys_name}_{n_samples}/data_{sys_name}_{n_samples}.csv",num_attributes=4, num_classes=4)

In [71]:
X

array([[ 2.1892 ,  2.4081 , -0.49971, -0.74031],
       [ 1.7434 ,  1.0898 , -0.10954, -0.41258],
       [ 1.0537 ,  0.13005,  0.93942, -0.14472],
       ...,
       [ 2.4862 ,  1.4989 , -0.75639,  0.54465],
       [ 1.352  ,  0.87072, -0.68432, -0.90102],
       [ 2.4397 ,  2.1971 , -0.22278, -0.79128]])

In [72]:
y

array([[ 2.1451 ,  2.3306 , -0.44092, -0.77409],
       [ 1.7419 ,  1.0445 , -0.01462, -0.45312],
       [ 1.1482 ,  0.11716,  0.94498, -0.1289 ],
       ...,
       [ 2.4205 ,  1.5353 , -0.65719,  0.36334],
       [ 1.3018 ,  0.78931, -0.50192, -0.81411],
       [ 2.4202 ,  2.1154 , -0.19575, -0.81685]])

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.5)
print(f" Shape of X_Training = {X_train.shape} \n Shape of X_Testing = {X_test.shape}")
print(f" Shape of Y_Training = {y_train.shape} \n Shape of Y_Testing = {y_test.shape}")


 Shape of X_Training = (249999, 4) 
 Shape of X_Testing = (250000, 4)
 Shape of Y_Training = (249999, 4) 
 Shape of Y_Testing = (250000, 4)


In [58]:
import numpy as np
from typing import Optional, List

class _Node:
    def __init__(self, depth: int = 0):
        self.depth = depth
        self.is_leaf = True
        self.feature: Optional[int] = None
        self.threshold: Optional[float] = None
        self.left: Optional[int] = None
        self.right: Optional[int] = None
        self.n_samples: int = 0
        self.value: Optional[np.ndarray] = None
        self.mse: float = 0.0
        self.idx: Optional[np.ndarray] = None  # keep track of samples reaching this node
        self.leaf_id: Optional[int] = None     # assign after pruning

class CustomDecisionTreeRegressor:
    def __init__(
        self,
        max_depth: Optional[int] = None,
        min_samples_split: int = 2,
        mse_threshold: float = 1e-7,
        min_improvement: float = 1e-7,
    ):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.mse_threshold = mse_threshold
        self.min_improvement = min_improvement

        self.n_features_in_: Optional[int] = None
        self.n_outputs_: Optional[int] = None
        self._nodes: List[_Node] = []

    @staticmethod
    def _mse(sum_y: np.ndarray, sum_y2: float, n: int) -> float:
        if n <= 0:
            return 0.0
        return float((sum_y2 - float(np.sum(sum_y ** 2)) / n) / n)

    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X)
        y = np.asarray(y)
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        self.n_features_in_ = X.shape[1]
        self.n_outputs_ = y.shape[1]
        self._nodes = []
        self._build_node(X, y, np.arange(X.shape[0]), depth=0)
        self._assign_leaf_ids()
        self._fitted_X = X
        self._fitted_y = y
        return self

    def _build_node(self, X, y, idx, depth):
        node_id = len(self._nodes)
        node = _Node(depth=depth)
        self._nodes.append(node)

        Y = y[idx]
        n_node = Y.shape[0]
        sum_y = Y.sum(axis=0)
        sum_y2 = float((Y ** 2).sum())

        node.n_samples = n_node
        node.value = sum_y / max(n_node, 1)
        node.mse = self._mse(sum_y, sum_y2, n_node)
        node.idx = idx

        stop_by_depth = (self.max_depth is not None and depth >= self.max_depth)
        if n_node < self.min_samples_split or stop_by_depth or node.mse <= self.mse_threshold:
            node.is_leaf = True
            return node_id

        best = self._best_split(X, y, idx, sum_y, sum_y2, n_node)
        if best is None:
            node.is_leaf = True
            return node_id

        feat, thr, left_idx, right_idx, mse_left, mse_right = best
        parent_mse = node.mse
        n_left, n_right = left_idx.size, right_idx.size
        weighted_child_mse = (n_left * mse_left + n_right * mse_right) / n_node
        gain = parent_mse - weighted_child_mse

        if gain <= self.min_improvement:
            node.is_leaf = True
            return node_id

        node.is_leaf = False
        node.feature = int(feat)
        node.threshold = float(thr)
        node.left = self._build_node(X, y, left_idx, depth + 1)
        node.right = self._build_node(X, y, right_idx, depth + 1)
        return node_id

    def _best_split(self, X, y, idx, sum_y, sum_y2, n_node):
        X_node = X[idx]
        Y_node = y[idx]
        total_sum_y = sum_y
        total_sum_y2 = sum_y2
        parent_mse = self._mse(total_sum_y, total_sum_y2, n_node)

        best_feat = None
        best_thr = None
        best_left_idx = None
        best_right_idx = None
        best_mse_left = None
        best_mse_right = None

        for f in range(self.n_features_in_):
            x = X_node[:, f]
            order = np.argsort(x, kind="mergesort")
            x_sorted = x[order]
            Y_sorted = Y_node[order]

            diffs = x_sorted[1:] - x_sorted[:-1]
            valid = diffs != 0.0
            if not np.any(valid):
                continue

            csum_y = np.cumsum(Y_sorted, axis=0)
            row_sq = np.einsum("ij,ij->i", Y_sorted, Y_sorted)
            csum_y2 = np.cumsum(row_sq)
            split_positions = np.nonzero(valid)[0]

            left_n = (split_positions + 1).astype(np.int64)
            right_n = n_node - left_n
            left_sum_y = csum_y[split_positions]
            right_sum_y = total_sum_y - left_sum_y
            left_sum_y2 = csum_y2[split_positions]
            right_sum_y2 = total_sum_y2 - left_sum_y2

            left_mse = (left_sum_y2 - np.sum(left_sum_y ** 2, axis=1) / left_n) / left_n
            right_mse = (right_sum_y2 - np.sum(right_sum_y ** 2, axis=1) / right_n) / right_n
            weighted_child_mse = (left_n * left_mse + right_n * right_mse) / n_node

            best_pos = int(np.argmin(weighted_child_mse))
            candidate_mse = float(weighted_child_mse[best_pos])
            if candidate_mse >= parent_mse:
                continue

            i = split_positions[best_pos]
            thr = 0.5 * (x_sorted[i] + x_sorted[i + 1])
            mask_left = x <= thr
            left_idx = idx[mask_left]
            right_idx = idx[~mask_left]
            if left_idx.size == 0 or right_idx.size == 0:
                continue

            if best_thr is None or candidate_mse < (best_mse_left + best_mse_right if best_mse_left is not None else np.inf):
                best_feat = f
                best_thr = thr
                best_mse_left = float(left_mse[best_pos])
                best_mse_right = float(right_mse[best_pos])
                best_left_idx = left_idx
                best_right_idx = right_idx

        if best_thr is None:
            return None
        return best_feat, best_thr, best_left_idx, best_right_idx, best_mse_left, best_mse_right

    # --------------------
    # Least squares & pruning by h
    # --------------------
    def _least_squares_solution(self, X_leaf, y_leaf):
        n_samples, n_features = X_leaf.shape
        X_aug = np.hstack([np.ones((n_samples, 1)), X_leaf])
        XtX = X_aug.T @ X_aug
        XtY = X_aug.T @ y_leaf
        Theta = np.linalg.pinv(XtX) @ XtY
        m0 = Theta[0, :]
        M = Theta[1:, :].T
        Y_hat = X_aug @ Theta
        residuals = y_leaf - Y_hat
        h_val = np.sum(residuals ** 2)
        return M, m0, h_val

    def prune_by_h(self, X, y, h_threshold: float = 0.1):
        X = np.asarray(X)
        y = np.asarray(y)
        if y.ndim == 1:
            y = y.reshape(-1, 1)

        def _prune_recursive(node_id):
            node = self._nodes[node_id]
            if node.is_leaf:
                return node.idx
            left_idx = _prune_recursive(node.left)
            right_idx = _prune_recursive(node.right)
            combined_idx = np.concatenate([left_idx, right_idx])
            _, _, combined_h = self._least_squares_solution(X[combined_idx], y[combined_idx])
            # prune if h <= h_threshold
            if combined_h <= h_threshold:
                node.is_leaf = True
                node.left = None
                node.right = None
                node.idx = combined_idx
                node.value = y[combined_idx].mean(axis=0)
            return node.idx if node.is_leaf else combined_idx

        _prune_recursive(0)
        self._assign_leaf_ids()

    # --------------------
    # Assign leaf IDs
    # --------------------
    def _assign_leaf_ids(self):
        counter = 0
        for node in self._nodes:
            if node.is_leaf:
                node.leaf_id = counter
                counter += 1

    # --------------------
    # Prediction
    # --------------------
    def predict_row(self, x):
        node_id = 0
        while not self._nodes[node_id].is_leaf:
            node = self._nodes[node_id]
            if x[node.feature] <= node.threshold:
                node_id = node.left
            else:
                node_id = node.right
        return self._nodes[node_id].value

    def predict(self, X):
        X = np.asarray(X)
        return np.vstack([self.predict_row(row) for row in X])

    # ---------------------------
    # Node stats
    # ---------------------------
    def print_node_sse(self):
        for i, node in enumerate(self._nodes):
            print(f"Node {i}, depth {node.depth}, samples {node.n_samples}, scaled SSE {node.sse:.6f}, leaf {node.is_leaf}")

    def count_nodes(self):
        return len(self._nodes)

    def count_leaves(self):
        return sum(1 for n in self._nodes if n.is_leaf)


In [59]:
# Train custom tree
tree = CustomDecisionTreeRegressor(mse_threshold=1.0)
tree.fit(X_train, y_train)
#tree.print_node_mse()

print("Predictions (first 5):", tree.predict(X_test[:5]))
print("Number of nodes:", tree.count_nodes())
print("Number of leaves:", tree.count_leaves())

Predictions (first 5): [[ 1.2978      2.9458      0.86112     0.69014   ]
 [ 0.1547      0.68649    -0.49733    -0.38349   ]
 [ 2.33363333  2.10146667  0.54932667 -0.63284333]
 [ 1.31876     2.43455    -0.06256525  0.56483   ]
 [ 0.43283     1.2009     -0.655      -0.53837   ]]
Number of nodes: 152599
Number of leaves: 76300


In [71]:
tree.prune_by_h(X_train, y_train, h_threshold=0.000001)

In [72]:
print("Predictions (first 5):", tree.predict(X_test[:5]))
print("Number of nodes:", tree.count_nodes())
print("Number of leaves:", tree.count_leaves())

Predictions (first 5): [[ 1.51231077  1.57459948  0.10930605  0.63443242]
 [ 1.08754056  1.13436    -0.01604778 -0.40725111]
 [ 1.58313604  1.50502192  0.11609518 -0.61576879]
 [ 1.39630515  1.34144703  0.05051873  0.54887167]
 [ 1.52160816  1.46163879  0.06171542 -0.47212175]]
Number of nodes: 152599
Number of leaves: 151604


In [92]:
def train_and_prune_COF_tree_v3(
    X_train, y_train, X_test=None, y_test=None,
    initial_tree_params=None, optimizer="gurobi",
    alpha=1e-6, h_min=0, ignore_h=False, n_jobs=-1
):
    """
    Train a decision tree, build COF models, and prune tree sequentially.
    """

    # -------------------------------
    # 1️⃣ Train initial tree
    # -------------------------------
    if initial_tree_params is None:
        initial_tree_params = {"max_depth": 5}

    tree = DecisionTreeRegressor(**initial_tree_params)
    tree.fit(X_train, y_train)

    # -------------------------------
    # 2️⃣ Build COF models for leaves in parallel
    # -------------------------------
    COF_model_tree = train_COF_on_leaves_parallel(
        X_train, y_train, tree, optimizer=optimizer, n_jobs=n_jobs
    )

    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    leaf_nodes = np.where(children_left == -1)[0]

    leaf_h_dict = {leaf: COF_model_tree[i]['CO_Model']['h'] for i, leaf in enumerate(leaf_nodes)}
    leaf_indices_dict = {leaf: COF_model_tree[i]['indices'] for i, leaf in enumerate(leaf_nodes)}
    leaf_COFS_dict = {leaf: COF_model_tree[i] for i, leaf in enumerate(leaf_nodes)}

    # -------------------------------
    # 3️⃣ Helper function: compute COF
    # -------------------------------
    def compute_h_and_COF(indices):
        """
        Returns h, M, m0 for given indices using optimizer
        """
        if ignore_h:
            return 0, np.zeros((1, X_train.shape[1])), np.zeros(1)

        if optimizer == "gurobi":
            M, m0, h = constrained_optimization_gurobi(X_train[indices], y_train[indices])
        elif optimizer == "gurobi_MSE":
            M, m0, h = constrained_optimization_MSE_gurobi(X_train[indices], y_train[indices])
        else:
            M, m0, h = constrained_optimization(X_train[indices], y_train[indices])

        return h, M, m0

    # -------------------------------
    # 4️⃣ Sequential pruning function
    # -------------------------------
    def prune_node(node):
        left = children_left[node]
        right = children_right[node]

        # Leaf node
        if left == -1 and right == -1:
            return leaf_h_dict[node], 1, leaf_indices_dict[node], False

        # Process children sequentially
        left_cost, left_leaves, left_indices, _ = prune_node(left)
        right_cost, right_leaves, right_indices, _ = prune_node(right)

        combined_indices = np.concatenate([left_indices, right_indices])
        subtree_cost = left_cost + right_cost
        subtree_leaves = left_leaves + right_leaves

        # Compute parent COF
        h_parent, M_parent, m0_parent = compute_h_and_COF(combined_indices)
        prune_cost = h_parent + alpha

        # Decide pruning
        prune_flag = (not ignore_h and h_parent < h_min) or (prune_cost <= subtree_cost)

        if prune_flag:
            # Update tree
            children_left[node] = -1
            children_right[node] = -1

            # Remove child leaves
            for child in [left, right]:
                leaf_COFS_dict.pop(child, None)
                leaf_h_dict.pop(child, None)
                leaf_indices_dict.pop(child, None)

            # Update parent leaf info
            leaf_h_dict[node] = h_parent
            leaf_indices_dict[node] = combined_indices
            leaf_COFS_dict[node] = {
                "leaf_id": node,
                "CO_Model": {"h": h_parent, "M": M_parent, "m0": m0_parent},
                "indices": combined_indices,
                "no_samples": len(combined_indices)
            }

            # Debug printing
            print(f"🌳 Pruning triggered at node {node}, h={h_parent:.6f}, alpha={alpha}")
            print(f"➡️ Leaf indices count: {len(combined_indices)}")
            print(f"➡️ M shape: {M_parent.shape}, m0 shape: {m0_parent.shape}")

            return prune_cost, 1, combined_indices, True
        else:
            return subtree_cost, subtree_leaves, combined_indices, False

    # -------------------------------
    # 5️⃣ Stats before pruning
    # -------------------------------
    num_leaves_before = len(leaf_nodes)
    print(f"Before pruning: Leaves={num_leaves_before}")
    print(f"h values before pruning: {list(leaf_h_dict.values())}")

    # -------------------------------
    # 6️⃣ Iterative pruning until stable
    # -------------------------------
    previous_leaf_count = -1
    while True:
        prune_node(0)
        COF_model_tree_pruned = list(leaf_COFS_dict.values())
        current_leaf_count = len(COF_model_tree_pruned)
        print(f"➡️ Current leaf count after step: {current_leaf_count}")
        if current_leaf_count == previous_leaf_count:
            break
        previous_leaf_count = current_leaf_count

    # -------------------------------
    # 7️⃣ Stats after pruning
    # -------------------------------
    print(f"After pruning: Leaves={len(COF_model_tree_pruned)}")
    print(f"h values after pruning: {[leaf['CO_Model']['h'] for leaf in COF_model_tree_pruned]}")

    return tree, COF_model_tree_pruned

In [77]:
tree = DecisionTreeRegressor(max_depth=5)
tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5)

In [93]:
tree, COF_model =train_and_prune_COF_tree_v3(X_train, y_train, X_test=X_test, y_test=y_test, 
                                initial_tree_params=None, optimizer="gurobi", 
                                alpha=2, h_min=1, ignore_h=False, n_jobs=-1)

Set parameter Username
Set parameter LicenseID to value 2598283
Academic license - for non-commercial use only - expires 2025-12-11
Set parameter Username
Set parameter LicenseID to value 2598283
Set parameter Username
Set parameter LicenseID to value 2598283
Set parameter Username
Set parameter Username
Set parameter LicenseID to value 2598283
Set parameter LicenseID to value 2598283
Academic license - for non-commercial use only - expires 2025-12-11
Academic license - for non-commercial use only - expires 2025-12-11
Set parameter Username
Set parameter LicenseID to value 2598283
Academic license - for non-commercial use only - expires 2025-12-11
Academic license - for non-commercial use only - expires 2025-12-11
Academic license - for non-commercial use only - expires 2025-12-11
Set parameter Username
Set parameter LicenseID to value 2598283
Set parameter Username
Set parameter LicenseID to value 2598283
Academic license - for non-commercial use only - expires 2025-12-11
Academic lic

In [90]:


def train_and_prune_COF_tree_v4(
    X_train, y_train, X_test=None, y_test=None,
    initial_tree_params=None, optimizer="gurobi",
    alpha=1e-6, h_min=0, ignore_h=False, n_jobs=-1
):
    """
    Train COF tree and prune bottom-up in parallel.
    """
    # 1️⃣ Train initial tree
    if initial_tree_params is None:
        initial_tree_params = {"max_depth": 5}
    tree = DecisionTreeRegressor(**initial_tree_params)
    tree.fit(X_train, y_train)

    # 2️⃣ Build COF models for leaves
    COF_model_tree = train_COF_on_leaves_parallel(
        X_train, y_train, tree, optimizer=optimizer, n_jobs=n_jobs
    )

    # 3️⃣ Leaf info mapping
    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    leaf_nodes = np.where(children_left == -1)[0]

    leaf_h_dict = {leaf: COF_model_tree[i]['CO_Model']['h'] for i, leaf in enumerate(leaf_nodes)}
    leaf_indices_dict = {leaf: COF_model_tree[i]['indices'] for i, leaf in enumerate(leaf_nodes)}
    leaf_COFS_dict = {leaf: COF_model_tree[i] for i, leaf in enumerate(leaf_nodes)}

    # -------------------------------
    # 4️⃣ Bottom-up parallel pruning
    # -------------------------------
    def prune_subtree(node):
        left = children_left[node]
        right = children_right[node]

        if left == -1 and right == -1:
            # Leaf node
            return node, leaf_h_dict[node], 1, leaf_indices_dict[node]

        # Process children in parallel
        results = Parallel(n_jobs=2)(
            delayed(prune_subtree)(child) for child in [left, right]
        )
        (left_node, left_h, left_leaves, left_indices), (right_node, right_h, right_leaves, right_indices) = results

        combined_indices = np.concatenate([left_indices, right_indices])
        subtree_cost = left_h + right_h
        subtree_leaves = left_leaves + right_leaves

        # Compute parent h
        if ignore_h:
            h_parent = 0
        else:
            if optimizer == "gurobi":
                _, _, h_parent = constrained_optimization_gurobi(X_train[combined_indices], y_train[combined_indices])
            elif optimizer == "gurobi_MSE":
                _, _, h_parent = constrained_optimization_MSE_gurobi(X_train[combined_indices], y_train[combined_indices])
            else:
                _, _, h_parent = constrained_optimization(X_train[combined_indices], y_train[combined_indices])

        prune_cost = h_parent + alpha
        prune_flag = (not ignore_h and h_parent < h_min) or (prune_cost <= subtree_cost)

        if prune_flag:
            # Prune children
            children_left[node] = -1
            children_right[node] = -1

            # Update tree and COF info
            tree.tree_.value[node] = np.array([[h_parent]])
            leaf_h_dict[node] = h_parent
            leaf_indices_dict[node] = combined_indices
            leaf_COFS_dict[node] = {
                "leaf_id": node,
                "CO_Model": {"h": h_parent},
                "indices": combined_indices,
                "no_samples": len(combined_indices)
            }

            # Remove pruned children from COF dict
            for ch in [left_node, right_node]:
                if ch in leaf_COFS_dict:
                    del leaf_COFS_dict[ch]

            print(f"Pruning triggered at node {node} with h={h_parent:.6f}")
            return node, prune_cost, 1, combined_indices
        else:
            return node, subtree_cost, subtree_leaves, combined_indices

    # -------------------------------
    # 5️⃣ Stats before pruning
    # -------------------------------
    h_values_before = list(leaf_h_dict.values())
    num_leaves_before = len(leaf_nodes)
    if X_test is not None and y_test is not None:
        nrmse_train_before = normalized_root_mean_square_error(y_train, tree.predict(X_train))
        nrmse_test_before = normalized_root_mean_square_error(y_test, tree.predict(X_test))
        nrmse_train_COF_before = normalized_root_mean_square_error(y_train, predict_from_COF(COF_model_tree, X_train, tree))
        nrmse_test_COF_before = normalized_root_mean_square_error(y_test, predict_from_COF(COF_model_tree, X_test, tree))
        print(f"Before pruning: Leaves={num_leaves_before}, NRMSE[Tree]={nrmse_train_before:.4f}/{nrmse_test_before:.4f}, "
              f"NRMSE[COF]={nrmse_train_COF_before:.4f}/{nrmse_test_COF_before:.4f}")
        print(f"h values before pruning: {h_values_before}")

    # -------------------------------
    # 6️⃣ Start bottom-up pruning
    # -------------------------------
    prune_subtree(0)
    COF_model_tree_pruned = list(leaf_COFS_dict.values())

    # -------------------------------
    # 7️⃣ Stats after pruning
    # -------------------------------
    leaf_h_dict_after = {leaf['leaf_id']: leaf['CO_Model']['h'] for leaf in COF_model_tree_pruned}
    h_values_after = list(leaf_h_dict_after.values())
    num_leaves_after = len(COF_model_tree_pruned)
    if X_test is not None and y_test is not None:
        nrmse_train_after = normalized_root_mean_square_error(y_train, tree.predict(X_train))
        nrmse_test_after = normalized_root_mean_square_error(y_test, tree.predict(X_test))
        nrmse_train_COF_after = normalized_root_mean_square_error(y_train, predict_from_COF(COF_model_tree_pruned, X_train, tree))
        nrmse_test_COF_after = normalized_root_mean_square_error(y_test, predict_from_COF(COF_model_tree_pruned, X_test, tree))
        print(f"After pruning: Leaves={num_leaves_after}, NRMSE[Tree]={nrmse_train_after:.4f}/{nrmse_test_after:.4f}, "
              f"NRMSE[COF]={nrmse_train_COF_after:.4f}/{nrmse_test_COF_after:.4f}")
        print(f"h values after pruning: {h_values_after}")

    return tree, COF_model_tree_pruned


🌳 Pruning triggered on node 14, h_parent=0.000010, alpha=2
➡️ Leaf indices count: 17341
➡️ M shape: (4, 4), m0 shape: (4,)
🌳 Pruning triggered on node 14, h_parent=0.000010, alpha=2
➡️ Leaf indices count: 17341
➡️ M shape: (4, 4), m0 shape: (4,)
🌳 Pruning triggered on node 35, h_parent=0.000289, alpha=2
➡️ Leaf indices count: 16166
➡️ M shape: (4, 4), m0 shape: (4,)
🌳 Pruning triggered on node 57, h_parent=0.000110, alpha=2
➡️ Leaf indices count: 15971
➡️ M shape: (4, 4), m0 shape: (4,)
🌳 Pruning triggered on node 35, h_parent=0.000289, alpha=2
➡️ Leaf indices count: 16166
➡️ M shape: (4, 4), m0 shape: (4,)
🌳 Pruning triggered on node 57, h_parent=0.000110, alpha=2
➡️ Leaf indices count: 15971
➡️ M shape: (4, 4), m0 shape: (4,)


In [89]:
tree, COF_model =train_and_prune_COF_tree_v4(X_train, y_train, X_test=X_test, y_test=y_test, 
                                initial_tree_params=None, optimizer="gurobi", 
                                alpha=2, h_min=1, ignore_h=False, n_jobs=-1)

NameError: name 'train_and_prune_COF_tree_v4' is not defined

In [7]:
import numpy as np
from dataclasses import dataclass
from typing import Optional, Dict, Tuple, List, Any

# Optional imports for per-leaf optimizers
try:
    import cvxpy as cp
except Exception:
    cp = None

try:
    import gurobipy as gp
    from gurobipy import GRB
except Exception:
    gp = None
    GRB = None


@dataclass
class _Node:
    # Split info
    feature: int = -1
    threshold: float = 0.0
    left: Optional[int] = None
    right: Optional[int] = None

    # Stats
    n_samples: int = 0
    sum_y: Optional[np.ndarray] = None       # shape (n_outputs,)
    sum_y2: float = 0.0                      # scalar: sum over samples of ||y||^2

    # Prediction at leaf
    value: Optional[np.ndarray] = None       # mean y, shape (n_outputs,)

    # Bookkeeping
    depth: int = 0
    is_leaf: bool = True
    leaf_id: Optional[int] = None

    # Cached pruning metric (computed when needed)
    _g_alpha: Optional[float] = None


class CustomDecisionTreeRegressor:
    """
    A fast, multi-output regression tree with:
    - MSE-based splits and stopping when leaf MSE (averaged across targets) ≤ threshold
    - max_depth stopping
    - post-pruning via cost-complexity (ccp_alpha)
    - min_leaf_samples enforced post-fit (does not influence split choices)
    - per-leaf constrained optimizers (cvxpy or gurobi) with prediction
    - utilities: predict, apply, get_leaf_indices, leaf_bounds (data-driven),
                 leaf_box_bounds (path constraints), score (R^2), prune, get_h
    """

    def __init__(
        self,
        max_depth: Optional[int] = None,
        mse_threshold: float = 0.0,              # stop when leaf MSE_avg ≤ threshold
        min_leaf_samples: Optional[int] = None,  # enforced post-fit only
        ccp_alpha: float = 0.0,                  # default no pruning at fit()
        random_state: Optional[int] = None,
        min_improvement: float = 0.0             # minimal SSE reduction required to split
    ):
        self.max_depth = max_depth
        self.mse_threshold = float(mse_threshold)
        self.min_leaf_samples = int(min_leaf_samples) if min_leaf_samples is not None else 0
        self.ccp_alpha = float(ccp_alpha)
        self.random_state = random_state
        self.min_improvement = float(min_improvement)

        # Fitted attributes
        self.n_features_in_: Optional[int] = None
        self.n_outputs_: Optional[int] = None
        self._nodes: List[_Node] = []
        self._root: Optional[int] = None
        self._fitted_X: Optional[np.ndarray] = None
        self._fitted_y: Optional[np.ndarray] = None

        # Per-leaf linear models from constrained optimization: leaf_id -> dict(M, m0, h, solver)
        self._leaf_models: Dict[int, Dict[str, Any]] = {}

        if self.random_state is not None:
            np.random.seed(self.random_state)

    # ---------------------------
    # Public API
    # ---------------------------

    def fit(self, X: np.ndarray, y: np.ndarray):
        self._validate_X_y(X, y)
        n_samples = X.shape[0]
        idx = np.arange(n_samples, dtype=np.int64)

        self._nodes = []
        self._root = self._build_node(X, y, idx, depth=0)
        self._assign_leaf_ids()

        # Optional pruning immediately after fit if ccp_alpha > 0
        if self.ccp_alpha > 0.0:
            self.prune(self.ccp_alpha)

        # Enforce min_leaf_samples post-fit without affecting split decisions
        if self.min_leaf_samples > 0:
            self._enforce_min_leaf_samples()

        # Re-assign leaf ids after any pruning
        self._assign_leaf_ids()

        # Cache training data (useful for utilities)
        self._fitted_X = X.copy()
        self._fitted_y = y.copy()
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        self._check_fitted()
        X = self._validate_X_only(X)
        preds = np.zeros((X.shape[0], self.n_outputs_), dtype=float)
        for i in range(X.shape[0]):
            node_idx = self._traverse(self._root, X[i])
            node = self._nodes[node_idx]
            preds[i] = node.value
        return preds

    def apply(self, X: np.ndarray) -> np.ndarray:
        self._check_fitted()
        X = self._validate_X_only(X)
        leaf_ids = np.empty(X.shape[0], dtype=np.int64)
        for i in range(X.shape[0]):
            node_idx = self._traverse(self._root, X[i])
            leaf_ids[i] = self._nodes[node_idx].leaf_id
        return leaf_ids

    def get_leaf_indices(self, X: Optional[np.ndarray] = None) -> Dict[int, np.ndarray]:
        """
        Returns mapping leaf_id -> sample indices that fall into that leaf.
        If X is None, uses the training data indices.
        """
        self._check_fitted()
        if X is None:
            X = self._fitted_X
            indices = np.arange(X.shape[0], dtype=np.int64)
        else:
            X = self._validate_X_only(X)
            indices = np.arange(X.shape[0], dtype=np.int64)

        leaf_ids = self.apply(X)
        mapping: Dict[int, List[int]] = {}
        for i, lid in enumerate(leaf_ids):
            mapping.setdefault(int(lid), []).append(int(indices[i]))
        return {lid: np.array(idxs, dtype=np.int64) for lid, idxs in mapping.items()}

    def leaf_bounds(self, X: Optional[np.ndarray] = None) -> Dict[int, Dict[str, np.ndarray]]:
        """
        Data-driven bounds: for each leaf, returns min/max for each feature among samples in that leaf.
        """
        self._check_fitted()
        if X is None:
            X = self._fitted_X
        X = self._validate_X_only(X)

        mapping = self.get_leaf_indices(X)
        bounds = {}
        for lid, idxs in mapping.items():
            Xi = X[idxs]
            bounds[lid] = {
                "min": Xi.min(axis=0),
                "max": Xi.max(axis=0),
            }
        return bounds

    def leaf_box_bounds(self, X: Optional[np.ndarray] = None) -> Dict[int, Dict[str, np.ndarray]]:
        """
        Returns path-based box constraints for each leaf, refined by actual data bounds.
        """
        self._check_fitted()
        if X is None:
            X = self._fitted_X
        X = self._validate_X_only(X)
        leaf_indices = self.get_leaf_indices(X)
    
        bounds = {}
    
        def dfs(nid: int, path: List[Tuple[int, float, str]]):
            node = self._nodes[nid]
            if node.is_leaf:
                lid = node.leaf_id
                idxs = leaf_indices.get(lid, [])
                if len(idxs) == 0:
                    return
                X_leaf = X[idxs]
                lower = X_leaf.min(axis=0)
                upper = X_leaf.max(axis=0)
                for f, thr, direction in path:
                    if direction == "left":
                        upper[f] = min(upper[f], thr)
                    else:
                        lower[f] = max(lower[f], np.nextafter(thr, np.inf))
                bounds[lid] = {"lower": lower, "upper": upper}
                return
            dfs(node.left, path + [(node.feature, node.threshold, "left")])
            dfs(node.right, path + [(node.feature, node.threshold, "right")])
    
        dfs(self._root, [])
        return bounds

    def score(self, X: np.ndarray, y: np.ndarray) -> float:
        """
        Multi-output R^2 averaged across outputs.
        """
        self._check_fitted()
        X = self._validate_X_only(X)
        y = self._validate_y_only(y)
        y_pred = self.predict(X)
        y_mean = y.mean(axis=0)
        ss_res = ((y - y_pred) ** 2).sum(axis=0)
        ss_tot = ((y - y_mean) ** 2).sum(axis=0)
        valid = ss_tot > 0
        if not np.any(valid):
            return 1.0
        r2_per_output = np.ones(self.n_outputs_, dtype=float)
        r2_per_output[valid] = 1.0 - (ss_res[valid] / ss_tot[valid])
        return float(r2_per_output.mean())

    def prune(self, ccp_alpha: float):
        """
        Cost-complexity post-pruning. Prunes all internal nodes whose g_alpha ≤ ccp_alpha.
        """
        self._check_fitted()
        alpha = float(ccp_alpha)
        if alpha <= 0.0:
            return

        changed = True
        while changed:
            _, _, prunable = self._compute_ccp_alphas(self._root)
            to_prune = [nid for (nid, g) in prunable if g <= alpha]
            changed = len(to_prune) > 0
            for nid in to_prune:
                self._prune_subtree_to_leaf(nid)
        self._assign_leaf_ids()

    def fit_leaf_optimizers(
        self,
        X: Optional[np.ndarray] = None,
        y: Optional[np.ndarray] = None,
        optimizer: str = "cvxpy",  # "cvxpy" or "gurobi"
        gurobi_params: Optional[Dict[str, Any]] = None
    ):
        """
        Train constrained optimization model on each leaf using samples that reach that leaf.
        Stores per-leaf M, m0, and h. Use predict_with_optimizers() to predict.
        """
        self._check_fitted()
        if X is None:
            X = self._fitted_X
        if y is None:
            y = self._fitted_y
        X = self._validate_X_only(X)
        y = self._validate_y_only(y)

        leaf_to_indices = self.get_leaf_indices(X)
        self._leaf_models = {}

        for lid, idxs in leaf_to_indices.items():
            if idxs.size == 0:
                continue
            X_leaf = X[idxs]
            y_leaf = y[idxs]
            if optimizer.lower() == "cvxpy":
                if cp is None:
                    raise ImportError("cvxpy is not available. Install cvxpy or choose optimizer='gurobi'.")
                M, m0, h = self._constrained_optimization_cvxpy(X_leaf, y_leaf)
                solver_name = "cvxpy"
            elif optimizer.lower() == "gurobi":
                if gp is None or GRB is None:
                    raise ImportError("gurobi is not available. Install gurobipy or choose optimizer='cvxpy'.")
                M, m0, h = self._constrained_optimization_gurobi(X_leaf, y_leaf, gurobi_params)
                solver_name = "gurobi"
            else:
                raise ValueError("optimizer must be 'cvxpy' or 'gurobi'.")

            self._leaf_models[int(lid)] = {"M": M, "m0": m0, "h": float(h), "solver": solver_name}

        return self

    def predict_with_optimizers(self, X: np.ndarray) -> np.ndarray:
        """
        Predict using per-leaf linear models (M, m0). Falls back to tree mean if a leaf has no model.
        """
        self._check_fitted()
        X = self._validate_X_only(X)
        y_pred = np.zeros((X.shape[0], self.n_outputs_), dtype=float)
        leaf_ids = self.apply(X)
        for i, lid in enumerate(leaf_ids):
            mdl = self._leaf_models.get(int(lid))
            if mdl is not None and mdl["M"] is not None and mdl["m0"] is not None:
                y_pred[i] = X[i] @ mdl["M"].T + mdl["m0"]
            else:
                node_idx = self._leaf_node_from_leaf_id(int(lid))
                y_pred[i] = self._nodes[node_idx].value
        return y_pred

    def get_h(self) -> Dict[int, float]:
        """
        Returns mapping leaf_id -> h from the constrained optimization.
        """
        return {int(lid): float(v["h"]) for lid, v in self._leaf_models.items() if "h" in v}

    # ---------------------------
    # Building and splitting
    # ---------------------------

    def _build_node(self, X: np.ndarray, y: np.ndarray, idx: np.ndarray, depth: int) -> int:
        node_id = len(self._nodes)
        node = _Node(depth=depth)
        self._nodes.append(node)

        # Stats for this node
        Y = y[idx]
        n_node = Y.shape[0]
        sum_y = Y.sum(axis=0)
        sum_y2 = float((Y ** 2).sum())
        node.n_samples = n_node
        node.sum_y = sum_y
        node.sum_y2 = sum_y2
        node.value = sum_y / max(n_node, 1)
    
        # Compute NRMSE for stopping
        y_pred = np.tile(node.value, (n_node, 1))
        y_range = np.max(Y) - np.min(Y)
        if y_range != 0:
            nrmse = np.sqrt(np.mean((Y - y_pred) ** 2)) / y_range
        else:
            nrmse = np.sqrt(np.sum((Y - y_pred) ** 2) / (n_node * self.n_outputs_))
    
        stop_by_depth = (self.max_depth is not None and depth >= self.max_depth)
        if n_node <= 1 or stop_by_depth or nrmse <= self.mse_threshold:
            node.is_leaf = True
            return node_id
        
        # Find best split
        best = self._best_split(X, y, idx, sum_y, sum_y2, n_node)
        if best is None:
            node.is_leaf = True
            return node_id

        feat, thr, left_idx, right_idx, sse_left, sse_right = best
        parent_sse = sse
        gain = parent_sse - (sse_left + sse_right)
        if gain <= self.min_improvement:
            node.is_leaf = True
            return node_id

        # Create children
        node.is_leaf = False
        node.feature = int(feat)
        node.threshold = float(thr)
        node.left = self._build_node(X, y, left_idx, depth + 1)
        node.right = self._build_node(X, y, right_idx, depth + 1)
        return node_id

    def _best_split(
        self,
        X: np.ndarray,
        y: np.ndarray,
        idx: np.ndarray,
        sum_y: np.ndarray,
        sum_y2: float,
        n_node: int
    ):
        X_node = X[idx]                                  # (n_node, n_features)
        Y_node = y[idx]                                  # (n_node, n_outputs)
        total_sum_y = sum_y
        total_sum_y2 = sum_y2

        best_feat = None
        best_thr = None
        best_sse_left = None
        best_sse_right = None
        best_left_idx = None
        best_right_idx = None
        parent_sse = self._sse(total_sum_y, total_sum_y2, n_node)

        # For each feature, compute optimal split via cumulative sums
        for f in range(self.n_features_in_):
            x = X_node[:, f]
            order = np.argsort(x, kind='mergesort')      # stable sort
            x_sorted = x[order]
            Y_sorted = Y_node[order]                     # (n_node, n_outputs)

            # Valid split positions where feature value changes
            diffs = x_sorted[1:] - x_sorted[:-1]
            valid = diffs != 0.0
            if not np.any(valid):
                continue

            # Cumulative sums for SSE computations
            csum_y = np.cumsum(Y_sorted, axis=0)                       # (n_node, n_outputs)
            row_sq = np.einsum('ij,ij->i', Y_sorted, Y_sorted)         # (n_node,)
            csum_y2 = np.cumsum(row_sq)                                 # (n_node,)

            split_positions = np.nonzero(valid)[0]  # indices i where split is between i and i+1

            left_n = (split_positions + 1).astype(np.int64)
            right_n = n_node - left_n

            left_sum_y = csum_y[split_positions]                        # (m, n_outputs)
            right_sum_y = total_sum_y - left_sum_y                      # (m, n_outputs)
            left_sum_y2 = csum_y2[split_positions]                      # (m,)
            right_sum_y2 = total_sum_y2 - left_sum_y2                   # (m,)

            # SSE for left/right: SSE = sum(y^2) - sum(y)^2 / n
            left_sse = left_sum_y2 - np.sum(left_sum_y ** 2, axis=1) / left_n
            right_sse = right_sum_y2 - np.sum(right_sum_y ** 2, axis=1) / right_n
            total_sse_after = left_sse + right_sse

            # Best position for this feature
            best_pos = int(np.argmin(total_sse_after))
            candidate_sse = float(total_sse_after[best_pos])
            if candidate_sse >= parent_sse:
                continue  # no gain

            # Candidate threshold: midpoint between two adjacent values
            i = split_positions[best_pos]
            thr = 0.5 * (x_sorted[i] + x_sorted[i + 1])

            # Derive indices for children (on original node data)
            mask_left = x <= thr
            left_idx = idx[mask_left]
            right_idx = idx[~mask_left]
            if left_idx.size == 0 or right_idx.size == 0:
                continue

            # Store if overall best
            if best_thr is None or candidate_sse < (best_sse_left + best_sse_right):
                best_feat = f
                best_thr = thr
                best_sse_left = float(left_sse[best_pos])
                best_sse_right = float(right_sse[best_pos])
                best_left_idx = left_idx
                best_right_idx = right_idx

        if best_thr is None:
            return None
        return best_feat, best_thr, best_left_idx, best_right_idx, best_sse_left, best_sse_right

    # ---------------------------
    # Pruning helpers (cost-complexity)
    # ---------------------------

    def _compute_ccp_alphas(self, nid: int) -> Tuple[float, int, List[Tuple[int, float]]]:
        node = self._nodes[nid]
        prunable: List[Tuple[int, float]] = []
        if node.is_leaf:
            R_T = self._node_impurity_as_leaf(node)
            return R_T, 1, prunable

        # Recurse
        R_left, L_left, P_left = self._compute_ccp_alphas(node.left)
        R_right, L_right, P_right = self._compute_ccp_alphas(node.right)
        prunable.extend(P_left)
        prunable.extend(P_right)

        R_T = R_left + R_right
        L_T = L_left + L_right

        # If we collapse node to a leaf:
        R_t = self._node_impurity_as_leaf(node)
        g = (R_t - R_T) / (L_T - 1.0) if L_T > 1 else np.inf
        node._g_alpha = g
        prunable.append((nid, g))
        return R_T, L_T, prunable

    def _prune_subtree_to_leaf(self, nid: int):
        node = self._nodes[nid]
        node.is_leaf = True
        node.left = None
        node.right = None
        node.feature = -1
        node.threshold = 0.0
        node._g_alpha = None

    def _node_impurity_as_leaf(self, node: _Node) -> float:
        n = node.n_samples
        if n == 0:
            return 0.0
        sse = self._sse(node.sum_y, node.sum_y2, n)
        # Use SSE (not averaged) to maintain additivity across subtree
        return sse

    def _enforce_min_leaf_samples(self):
        if self.min_leaf_samples <= 0:
            return

        def postorder(nid: int) -> int:
            node = self._nodes[nid]
            if node.is_leaf:
                return node.n_samples
            ln = postorder(node.left)
            rn = postorder(node.right)

            left_node = self._nodes[node.left]
            right_node = self._nodes[node.right]
            need_prune = False
            if left_node.is_leaf and left_node.n_samples < self.min_leaf_samples:
                need_prune = True
            if right_node.is_leaf and right_node.n_samples < self.min_leaf_samples:
                need_prune = True

            if need_prune:
                self._prune_subtree_to_leaf(nid)
                return node.n_samples
            return ln + rn

        postorder(self._root)

    # ---------------------------
    # Per-leaf constrained optimization
    # ---------------------------

    def _constrained_optimization_cvxpy(self, X_leaf: np.ndarray, y_leaf: np.ndarray):
        n_samples, n_features = X_leaf.shape
        n_outputs = y_leaf.shape[1]

        M = cp.Variable((n_outputs, n_features))
        m0 = cp.Variable((n_outputs,))
        h = cp.Variable(nonneg=True)

        prediction = X_leaf @ M.T + m0
        constraint_expr = cp.sum_squares(prediction - y_leaf)
        objective = cp.Minimize(h)
        constraints = [constraint_expr <= h]

        prob = cp.Problem(objective, constraints)
        try:
            prob.solve(solver=cp.SCS, verbose=False)
        except Exception:
            prob.solve(verbose=False)

        Mv = M.value if M.value is not None else None
        m0v = m0.value if m0.value is not None else None
        hv = h.value if h.value is not None else np.inf
        return Mv, m0v, float(hv)

    def _constrained_optimization_gurobi(
        self,
        X_leaf: np.ndarray,
        y_leaf: np.ndarray,
        gurobi_params: Optional[Dict[str, Any]] = None
    ):
        n_samples, n_features = X_leaf.shape
        n_outputs = y_leaf.shape[1]

        model = gp.Model("constrained_optimization")
        model.setParam("OutputFlag", 0)
        if gurobi_params:
            for k, v in gurobi_params.items():
                model.setParam(k, v)

        # Decision variables
        M = model.addVars(n_outputs, n_features, lb=-GRB.INFINITY, name="M")
        m0 = model.addVars(n_outputs, lb=-GRB.INFINITY, name="m0")
        h = model.addVar(lb=0.0, name="h")

        # Residuals squared sum
        quad_expr = gp.QuadExpr()
        for i in range(n_samples):
            for k in range(n_outputs):
                expr = m0[k]
                for j in range(n_features):
                    expr = expr + M[k, j] * float(X_leaf[i, j])
                diff = expr - float(y_leaf[i, k])
                quad_expr.add(diff * diff)

        model.addQConstr(quad_expr <= h, name="residual_bound")
        model.setObjective(h, GRB.MINIMIZE)
        model.optimize()

        if model.status in [GRB.OPTIMAL, GRB.SUBOPTIMAL] or model.SolCount > 0:
            M_val = np.array([[M[k, j].X for j in range(n_features)] for k in range(n_outputs)], dtype=float)
            m0_val = np.array([m0[k].X for k in range(n_outputs)], dtype=float)
            h_val = float(h.X)
        else:
            M_val, m0_val, h_val = None, None, np.inf

        return M_val, m0_val, h_val

    # ---------------------------
    # Traversal and utilities
    # ---------------------------

    def _assign_leaf_ids(self):
        counter = 0

        def dfs(nid: int):
            nonlocal counter
            node = self._nodes[nid]
            if node.is_leaf:
                node.leaf_id = counter
                counter += 1
            else:
                dfs(node.left)
                dfs(node.right)

        dfs(self._root)

    def _leaf_node_from_leaf_id(self, leaf_id: int) -> int:
        for i, node in enumerate(self._nodes):
            if node.is_leaf and node.leaf_id == leaf_id:
                return i
        raise KeyError(f"Leaf id {leaf_id} not found.")

    def _traverse(self, nid: int, x: np.ndarray) -> int:
        node = self._nodes[nid]
        while not node.is_leaf:
            if x[node.feature] <= node.threshold:
                nid = node.left
            else:
                nid = node.right
            node = self._nodes[nid]
        return nid

    @staticmethod
    def _sse(sum_y: np.ndarray, sum_y2: float, n: int) -> float:
        if n <= 0:
            return 0.0
        # SSE across outputs = sum(y^2) - sum(y)^2 / n
        return float(sum_y2 - float(np.sum(sum_y ** 2)) / n)

    # ---------------------------
    # Validation and checks
    # ---------------------------

    def _validate_X_y(self, X: np.ndarray, y: np.ndarray):
        if not isinstance(X, np.ndarray) or not isinstance(y, np.ndarray):
            raise TypeError("X and y must be NumPy arrays.")
        if X.ndim != 2:
            raise ValueError("X must be 2D with shape (n_samples, n_features).")
        if y.ndim != 2:
            raise ValueError("y must be 2D with shape (n_samples, n_targets).")
        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must have the same number of samples.")
        if not np.issubdtype(X.dtype, np.number) or not np.issubdtype(y.dtype, np.number):
            raise TypeError("X and y must be numeric.")
        if not np.isfinite(X).all() or not np.isfinite(y).all():
            raise ValueError("X and y must be finite (no NaNs or infs).")

        self.n_features_in_ = X.shape[1]
        self.n_outputs_ = y.shape[1]

        # Ensure contiguous float arrays
        if X.dtype != np.float64:
            X = X.astype(np.float64, copy=False)
        if y.dtype != np.float64:
            y = y.astype(np.float64, copy=False)

        # Store back (caller passes references)
        # No return needed; callers already hold X, y

    def _validate_X_only(self, X: np.ndarray) -> np.ndarray:
        if not isinstance(X, np.ndarray):
            raise TypeError("X must be a NumPy array.")
        if X.ndim != 2:
            raise ValueError("X must be 2D with shape (n_samples, n_features).")
        if self.n_features_in_ is None:
            raise RuntimeError("Model is not fitted yet.")
        if X.shape[1] != self.n_features_in_:
            raise ValueError(f"X has {X.shape[1]} features, expected {self.n_features_in_}.")
        if not np.issubdtype(X.dtype, np.number):
            raise TypeError("X must be numeric.")
        if not np.isfinite(X).all():
            raise ValueError("X must be finite (no NaNs or infs).")
        if X.dtype != np.float64:
            X = X.astype(np.float64, copy=False)
        return X

    def _validate_y_only(self, y: np.ndarray) -> np.ndarray:
        if not isinstance(y, np.ndarray):
            raise TypeError("y must be a NumPy array.")
        if y.ndim != 2:
            raise ValueError("y must be 2D with shape (n_samples, n_targets).")
        if self.n_outputs_ is None:
            raise RuntimeError("Model is not fitted yet.")
        if y.shape[1] != self.n_outputs_:
            raise ValueError(f"y has {y.shape[1]} targets, expected {self.n_outputs_}.")
        if not np.issubdtype(y.dtype, np.number):
            raise TypeError("y must be numeric.")
        if not np.isfinite(y).all():
            raise ValueError("y must be finite (no NaNs or infs).")
        if y.dtype != np.float64:
            y = y.astype(np.float64, copy=False)
        return y

    def _check_fitted(self):
        if self._root is None or self.n_features_in_ is None or self.n_outputs_ is None:
            raise RuntimeError("Estimator is not fitted yet.")


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score



# Initialize and train the tree
tree = CustomDecisionTreeRegressor(mse_threshold=0.5)

tree.fit(X_train, y_train) 
#max_depth=5,
#mse_threshold=0.05,
#min_leaf_samples=10,
#ccp_alpha=0.0,
#random_state=42
#tree.fit(X_train, y_train)

# Evaluate tree predictions
y_pred_tree = tree.predict(X_test)


r2_tree = r2_score(y_test, y_pred_tree, multioutput='uniform_average')
print(f"Tree R² score: {r2_tree:.4f}")

# Fit constrained optimizers on leaves
tree.fit_leaf_optimizers(optimizer="gurobi")

# Predict using leaf optimizers
y_pred_opt = tree.predict_with_optimizers(X_test)
r2_opt = r2_score(y_test, y_pred_opt, multioutput='uniform_average')
print(f"Optimizer R² score: {r2_opt:.4f}")

# Get leaf IDs and bounds
leaf_ids = tree.apply(X_test)
leaf_indices = tree.get_leaf_indices(X_test)
leaf_bounds = tree.leaf_bounds(X_test)
leaf_box_bounds = tree.leaf_box_bounds()
leaf_h_values = tree.get_h()

# Print summary
print(f"Number of leaves: {len(leaf_indices)}")
for lid in sorted(leaf_indices):
    print(f"Leaf {lid}: samples={len(leaf_indices[lid])}, h={leaf_h_values.get(lid, 'N/A'):.4f}")
    print(f"  Data bounds: min={leaf_bounds[lid]['min']}, max={leaf_bounds[lid]['max']}")
    print(f"  Box bounds: lower={leaf_box_bounds[lid]['lower']}, upper={leaf_box_bounds[lid]['upper']}")


In [75]:
# Get and print h values
h_values = tree.get_h()
print("\n📊 h values from constrained optimization per leaf:")
for lid in sorted(h_values):
    print(f"Leaf {lid}: h = {h_values[lid]:.6f}")



📊 h values from constrained optimization per leaf:
Leaf 0: h = 1505.394794


In [12]:
tree._nodes

[_Node(feature=-1, threshold=0.0, left=None, right=None, n_samples=249999, sum_y=array([377024.70385036, 373585.80729221,  18115.54099502, -13792.22580622]), sum_y2=1594698.9191638913, value=array([ 1.50810485,  1.49434921,  0.07246245, -0.05516912]), depth=0, is_leaf=True, leaf_id=0, _g_alpha=None)]

In [48]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

def bottom_up_tree_debug(tree, X_train, y_train):
    """
    Iterate tree bottom-up, printing node info (supports multi-output).
    """
    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    node_count = tree.tree_.node_count

    # Step 1: assign leaf indices for all samples
    sample_leaves = tree.apply(X_train)
    node_indices = {i: np.where(sample_leaves == i)[0] for i in range(node_count)}

    # Step 2: recursively collect indices for all nodes
    def collect_indices(node):
        left = children_left[node]
        right = children_right[node]

        if left == -1 and right == -1:
            # Leaf: already stored
            return node_indices[node]

        # Internal node: collect from children
        left_indices = collect_indices(left)
        right_indices = collect_indices(right)
        combined = np.concatenate([left_indices, right_indices])
        node_indices[node] = combined
        return combined

    collect_indices(0)  # populate all nodes

    # Step 3: Bottom-up traversal
    def bottom_up(node):
        left = children_left[node]
        right = children_right[node]

        # Process children first
        if left != -1:
            bottom_up(left)
        if right != -1:
            bottom_up(right)

        indices = node_indices[node]
        node_value = tree.tree_.value[node].flatten()  # may be >1D if multi-output

        # Compute error properly for multi-output
        if len(indices) > 0:
            preds = np.tile(node_value, (len(indices), 1))  # repeat value for samples
            true_vals = y_train[indices]
            error = np.mean(np.sum((true_vals - preds) ** 2, axis=1))
        else:
            error = np.nan

        print(f"Node index: {node}")
        print(f"Training indices on node: {indices}")
        print(f"Node value: {node_value}")
        print(f"MSE error at node: {error:.4f}\n")

    bottom_up(0)


In [49]:
# Example usage
tree = DecisionTreeRegressor(max_depth=3).fit(X_train, y_train)
bottom_up_tree_debug(tree, X_train, y_train)

Node index: 3
Training indices on node: [     5      9     12 ... 449972 449994 449998]
Node value: [ 0.55406066  0.65800726 -0.3498799  -0.03258353]
MSE error at node: 0.6105

Node index: 4
Training indices on node: [     6      7     17 ... 449990 449993 449997]
Node value: [ 0.64185204  0.65272199  0.53929005 -0.02492924]
MSE error at node: 0.6070

Node index: 2
Training indices on node: [     5      9     12 ... 449990 449993 449997]
Node value: [ 0.59770144  0.65537997  0.0921233  -0.02877861]
MSE error at node: 0.8083

Node index: 6
Training indices on node: [    11     16     29 ... 449980 449988 449995]
Node value: [ 1.73530217  0.772283    0.08735064 -0.49267212]
MSE error at node: 0.6963

Node index: 7
Training indices on node: [     2      3     14 ... 449984 449986 449991]
Node value: [1.73734368 0.8609972  0.09517587 0.3965426 ]
MSE error at node: 0.7008

Node index: 5
Training indices on node: [    11     16     29 ... 449984 449986 449991]
Node value: [ 1.73632069  0.816

In [23]:


def train_and_prune_COF_tree_v2(X_train, y_train, X_test=None, y_test=None, 
                                initial_tree_params=None, optimizer="gurobi", 
                                alpha=1e-6, h_min=0, ignore_h=False, n_jobs=-1):
    """
    Train, build COF models, prune in parallel, and print stats.
    """
    """
    Train a DecisionTreeRegressor, build COF models for leaves, and prune the tree in place.

    Parameters
    ----------
    X_train, y_train : np.ndarray
        Training data
    initial_tree_params : dict
        Parameters to initialize DecisionTreeRegressor
    optimizer : str
        COF optimizer ("gurobi", "gurobi_MSE", etc.)
    alpha : float
        Penalty for number of leaves
    h_min : float
        Minimum allowed h for pruning
    ignore_h : bool
        If True, h is ignored in pruning
    n_jobs : int
        Number of parallel jobs for leaf computation

    Returns
    -------
    tree : DecisionTreeRegressor
        The pruned tree
    COF_model_tree : list of dict
        COF models (updated after pruning)
    """
    # 1️⃣ Train initial tree
    if initial_tree_params is None:
        initial_tree_params = {"max_depth": 5}
    tree = DecisionTreeRegressor(**initial_tree_params)
    tree.fit(X_train, y_train)

    # 2️⃣ Build COF models for leaves
    COF_model_tree = train_COF_on_leaves_parallel(X_train, y_train, tree, optimizer=optimizer, n_jobs=n_jobs)

    # 3️⃣ Leaf info mapping
    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    leaf_nodes = np.where(children_left == -1)[0]
    leaf_h_dict = {leaf: COF_model_tree[i]['CO_Model']['h'] for i, leaf in enumerate(leaf_nodes)}
    leaf_indices_dict = {leaf: COF_model_tree[i]['indices'] for i, leaf in enumerate(leaf_nodes)}
    leaf_COFS_dict = {leaf: COF_model_tree[i] for i, leaf in enumerate(leaf_nodes)}

    # Helper function to compute h dynamically
    def compute_h(indices):
        if ignore_h:
            return 0
        if optimizer == "gurobi":
            _, _, h = constrained_optimization_gurobi(X_train[indices], y_train[indices])
        elif optimizer == "gurobi_MSE":
            _, _, h = constrained_optimization_MSE_gurobi(X_train[indices], y_train[indices])
        else:
            _, _, h = constrained_optimization(X_train[indices], y_train[indices])
        return max(h, h_min)

    # -------------------------------
    # 4️⃣ Parallelized recursive pruning
    # -------------------------------
    def prune_node(node):
        left = children_left[node]
        right = children_right[node]

        if left == -1 and right == -1:
            # Leaf
            h_leaf = leaf_h_dict[node]
            return h_leaf, 1, leaf_indices_dict[node]

        # Evaluate left/right subtrees in parallel
        results = Parallel(n_jobs=2)(
            delayed(prune_node)(child) for child in [left, right]
        )
        (left_cost, left_leaves, left_indices), (right_cost, right_leaves, right_indices) = results

        combined_indices = np.concatenate([left_indices, right_indices])
        subtree_cost = left_cost + right_cost
        subtree_leaves = left_leaves + right_leaves

        h_parent = compute_h(combined_indices)
        prune_cost = h_parent + alpha
        prune_leaves = 1

        if prune_cost <= subtree_cost:
            # Prune children
            children_left[node] = -1
            children_right[node] = -1

            # Update tree.value
            tree.tree_.value[node] = np.array([[h_parent]])
            leaf_h_dict[node] = h_parent
            leaf_indices_dict[node] = combined_indices

            leaf_COFS_dict[node] = {
                "leaf_id": node,
                "CO_Model": {"h": h_parent},
                "indices": combined_indices,
                "no_samples": len(combined_indices)
            }
            return prune_cost, prune_leaves, combined_indices
        else:
            return subtree_cost, subtree_leaves, combined_indices

    # 5️⃣ Stats before pruning
    num_leaves_before = len(leaf_nodes)
    if X_test is not None and y_test is not None:
        nrmse_train_before = normalized_root_mean_square_error(y_train, tree.predict(X_train))
        nrmse_test_before = normalized_root_mean_square_error(y_test, tree.predict(X_test))
        nrmse_train_COF_before = normalized_root_mean_square_error(y_train, predict_from_COF(COF_model_tree, X_train, tree))
        nrmse_test_COF_before = normalized_root_mean_square_error(y_test, predict_from_COF(COF_model_tree, X_test, tree))
        print(f"Before pruning: Leaves={num_leaves_before}, NRMSE[Tree]={nrmse_train_before:.4f}/{nrmse_test_before:.4f}, NRMSE[COF]={nrmse_train_COF_before:.4f}/{nrmse_test_COF_before:.4f}")

    # 6️⃣ Start pruning from root
    prune_node(0)

    # 7️⃣ Stats after pruning
    COF_model_tree_pruned = list(leaf_COFS_dict.values())
    num_leaves_after = len(COF_model_tree_pruned)
    if X_test is not None and y_test is not None:
        nrmse_train_after = normalized_root_mean_square_error(y_train, tree.predict(X_train))
        nrmse_test_after = normalized_root_mean_square_error(y_test, tree.predict(X_test))
        nrmse_train_COF_after = normalized_root_mean_square_error(y_train, predict_from_COF(COF_model_tree_pruned, X_train, tree))
        nrmse_test_COF_after = normalized_root_mean_square_error(y_test, predict_from_COF(COF_model_tree_pruned, X_test, tree))
        print(f"After pruning: Leaves={num_leaves_after}, NRMSE[Tree]={nrmse_train_after:.4f}/{nrmse_test_after:.4f}, NRMSE[COF]={nrmse_train_COF_after:.4f}/{nrmse_test_COF_after:.4f}")

    return tree, COF_model_tree_pruned


In [28]:
tree, COF_model =train_and_prune_COF_tree_v2(X_train, y_train, X_test=X_test, y_test=y_test, 
                                initial_tree_params=None, optimizer="gurobi", 
                                alpha=2, h_min=1, ignore_h=False, n_jobs=-1)

Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license -

In [27]:
get_h_from_COF(COF_model)

[15.247495125548335,
 15.970116469724417,
 9.896257219805027e-05,
 13.297375726471527,
 10.063166303461312,
 19.84039996511953,
 0.0002915225348566648,
 26.351826120993845,
 19.731365798253936,
 27.784298026201956,
 0.020334491560324075,
 2.8069198481649876e-05,
 11.76965344336756,
 12.840628849843235,
 19.50031001173883,
 12.95491713564067,
 26.714420813368676,
 19.631612981318483,
 2.882897122440768e-05,
 12.976188609804481,
 15.59830223027343,
 5.081453184122862e-05,
 3.616854578928122e-05,
 5.152554209046204e-05,
 0.00012612581272515916,
 12.577742652167915,
 1.3682289603790343e-05,
 3.816671606296249e-05,
 26.894228586387467,
 12.080145465898758,
 0.00013950234297040176,
 15.719675279783754]

In [8]:
numLeaves = tree.get_n_leaves()
print(f"Number of leaves: {tree.get_n_leaves()}")
print(f"Total depth of tree: {tree.get_depth()}")
print(f"Number of nodes: {tree.tree_.node_count}")


Number of leaves: 32
Total depth of tree: 5
Number of nodes: 63


In [ ]:
# Get pruning path
path = tree.cost_complexity_pruning_path(X, y)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Train trees for each alpha
trees = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
    clf.fit(X, y)
    trees.append(clf)

# Evaluate on validation set and pick best

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 1. Train/Val/Test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 2. Train full tree and get pruning path
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

path = tree.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas

# 3. Train/prune with different alphas and evaluate on validation set
val_mse = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict(X_val)
    val_mse.append(mean_squared_error(y_val, y_val_pred))

# 4. Find best alpha
best_alpha = ccp_alphas[np.argmin(val_mse)]

# 5. Retrain final tree on Train+Val
X_trainval = np.vstack([X_train, X_val])
y_trainval = np.hstack([y_train, y_val])

final_tree = DecisionTreeRegressor(random_state=42, ccp_alpha=best_alpha)
final_tree.fit(X_trainval, y_trainval)

# 6. Test evaluation
y_test_pred = final_tree.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)

print("Best alpha:", best_alpha)
print("Test MSE:", test_mse)

# 7. Plot validation curve
plt.figure(figsize=(8, 5))
plt.plot(ccp_alphas, val_mse, marker="o", drawstyle="steps-post")
plt.axvline(best_alpha, color="red", linestyle="--", label=f"Best α = {best_alpha:.5f}")
plt.xlabel("ccp_alpha (complexity parameter)")
plt.ylabel("Validation MSE")
plt.title("Validation Curve for Cost Complexity Pruning")
plt.legend()
plt.show()


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed

# 1. Train/Val/Test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 2. Get pruning path
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)
path = tree.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas

# 3. Parallel training & validation
def evaluate_alpha(ccp_alpha):
    clf = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict(X_val)
    return mean_squared_error(y_val, y_val_pred)

val_mse = Parallel(n_jobs=-1)(delayed(evaluate_alpha)(alpha) for alpha in ccp_alphas)

# 4. Pick best alpha
best_alpha = ccp_alphas[np.argmin(val_mse)]

# 5. Retrain final tree on Train+Val
X_trainval = np.vstack([X_train, X_val])
y_trainval = np.hstack([y_train, y_val])

final_tree = DecisionTreeRegressor(random_state=42, ccp_alpha=best_alpha)
final_tree.fit(X_trainval, y_trainval)

# 6. Test evaluation
y_test_pred = final_tree.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)

print("Best alpha:", best_alpha)
print("Test MSE:", test_mse)

# 7. Plot validation curve
plt.figure(figsize=(8, 5))
plt.plot(ccp_alphas, val_mse, marker="o", drawstyle="steps-post")
plt.axvline(best_alpha, color="red", linestyle="--", label=f"Best α = {best_alpha:.5f}")
plt.xlabel("ccp_alpha (complexity parameter)")
plt.ylabel("Validation MSE")
plt.title("Validation Curve for Cost Complexity Pruning")
plt.legend()
plt.show()


KeyboardInterrupt: 

In [ ]:
print("Best alpha:", best_alpha)
print("Test MSE:", test_mse)